In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 불러오기

In [20]:
file_path = "C:/dev/study/Python_Project/chap2/data/"

df = pd.read_csv(f"{file_path}Customer-Churn-Records.csv",sep = ',')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [21]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Complain',
       'Satisfaction Score', 'Card Type', 'Point Earned'],
      dtype='object')

In [22]:
df.shape

(10000, 18)

## 2. 결측치 확인

In [23]:
# 결측치 없음
print((df.isnull().sum()/len(df)).sum())
df.info()

0.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  C

## 3. EDA

### 팀장님(장이건 님) 코드 사용

In [24]:
# import numpy as np

# corr = df.select_dtypes(include=['number']).drop(columns=['RowNumber', 'CustomerId']).corr().round(2)
# plt.figure(figsize = (10,5))
# sns.heatmap(corr, annot = True, cmap = 'coolwarm')

- CreditScore : 신용점수
- Age : 나이
- Tenure : 거래기간
- Balance : 잔액
- NumOfProducts : 제품 보유량 --> 1~4까지, 범주형으로 취급 가능 -> 1,2,3이상으로
- EstimateSalary : 급여 수준
- Satisfaction Score : 만족도 --> 1~5, 범주형으로 취급 가능 -> 점수라서 수치형으로 유지
- Point Earned : 포인트

------------------------------------------------

- Geography : 지역
- Gender : 성별
- Card Type : 카드 등급
- Complain : 컴플레인 유무
- IsActiveMember : 활성/비활성 고객
- HasCrCard : 신용카드 보유 유무

### feature별 exited 값과의 연관성

In [25]:
from scipy.stats import chi2_contingency

for col in df.columns:
    table = pd.crosstab(df[col], df['Exited'])
    chi2, p, dof, expected = chi2_contingency(table)
    print(col,":", p<0.05)

RowNumber : False
CustomerId : False
Surname : False
CreditScore : False
Geography : True
Gender : True
Age : True
Tenure : False
Balance : True
NumOfProducts : True
HasCrCard : False
IsActiveMember : True
EstimatedSalary : False
Exited : True
Complain : True
Satisfaction Score : False
Card Type : False
Point Earned : False


In [26]:
# 불필요 column 제거

df_drop = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [27]:
# 타겟 불균형치
target_ratio = df_drop["Exited"].value_counts(normalize=True)*100
print(target_ratio)

Exited
0    79.62
1    20.38
Name: proportion, dtype: float64


## 4. 이상치 확인

In [28]:
# numeric_cols = df_drop.drop(columns=["Exited"]).select_dtypes(include=np.number).columns

# n_cols = 3
# n_rows = int(np.ceil(len(numeric_cols) / n_cols))

# fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*4))

# # axes를 1차원 배열로 변환해서 루프 돌리기
# axes = axes.flatten()

# for i, col in enumerate(numeric_cols):
#     axes[i].boxplot(df_drop[col].dropna())
#     axes[i].set_title(col)

# # 남는 subplot은 숨김 처리
# for j in range(i+1, len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()

### Age 이상치 제거

In [29]:
# Q1 = df_drop['Age'].quantile(0.25)
# Q3 = df_drop['Age'].quantile(0.75)
# IQR = Q3 - Q1

# # 이상치 기준
# lower = Q1 - 1.5 * IQR
# upper = Q3 + 1.5 * IQR

# # 필터링
# df_clean = df_drop[(df_drop['Age'] >= lower) & (df_drop['Age'] <= upper)].copy()


In [30]:
# 가입 상품 4개 이상은 희귀 --> 3개 이상을 묶음
# 별로 영향 없었음
df_clean = df_drop.copy()
df_clean["NumOfProducts_"] = df_clean["NumOfProducts"].apply(
    lambda x: str(x) if x < 3 else "3_more"
)

In [31]:
# 잔액 정규화
# 별로 영향 없었음

# df_clean["Balance_log"] = np.log1p(df_clean["Balance"])

In [32]:
# drop_cols = ['Exited']

# # 범주형 컬럼
# cat_cols = list(df_clean.select_dtypes(include=['object','category']).columns) \
#     + ['Complain','IsActiveMember','HasCrCard']

# # 수치형 컬럼
# num_cols = [col for col in df_clean.columns if col not in cat_cols + drop_cols]

# # ---------- 4열 subplot ----------
# n_cols = 4
# n_rows = int(np.ceil(len(num_cols) / n_cols))

# fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*4))
# axes = axes.flatten()

# for i, col in enumerate(num_cols):
#     try:
#         df_clean['bin'] = pd.qcut(df_clean[col], q=5, duplicates='drop')
#         churn_rate = pd.crosstab(df_clean['bin'], df_clean['Exited'], normalize='index') * 100
#         churn_rate[1].plot(kind='bar', color='salmon', ax=axes[i])
#         axes[i].set_title(f"{col} bins (Exited=1)")
#         axes[i].set_ylabel("Churn Rate (%)")
#         axes[i].set_xlabel(col)
#         axes[i].tick_params(axis='x', rotation=30)
#     except ValueError:
#         axes[i].text(0.5, 0.5, "Not enough unique values",\
#             ha='center', va='center')
#         axes[i].set_title(f"{col} (skipped)")
#         axes[i].set_xticks([])
#         axes[i].set_yticks([])

# # 남는 subplot 제거
# for j in range(i+1, len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()

강한 영향: Age (나이가 많을수록 ↑), NumOfProducts (3개 이상이면 폭증)

중간 영향: CreditScore (낮으면 ↑), Balance (특정 구간 ↑)

약한 영향/무의미: Tenure, EstimatedSalary, Satisfaction Score, Point Earned

In [33]:
# # 범주형 컬럼
# cat_cols = [c for c in (list(df_clean.select_dtypes(include=['object','category']).columns) 
#                         + ['Complain','IsActiveMember','HasCrCard']) 
#             if c != 'Surname']

# n_cols = 3
# n_rows = int(np.ceil(len(cat_cols) / n_cols))

# fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*4))
# axes = axes.flatten()

# for i, col in enumerate(cat_cols):
#     churn_rate = pd.crosstab(df_clean[col], df_clean['Exited'], normalize='index') * 100
#     churn_rate[1].plot(kind='bar', color='skyblue', ax=axes[i])
#     axes[i].set_title(f"{col} (Exited=1)")
#     axes[i].set_ylabel("Churn Rate (%)")
#     axes[i].set_xlabel(col)
#     axes[i].tick_params(axis='x', rotation=30)

# # 남는 subplot 제거
# for j in range(i+1, len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()

- 강한 영향 변수
    - Complain: 불만 제기 고객 = 이탈 거의 확정적
    - IsActiveMember: 활성 여부에 따라 이탈률 절반 수준 차이
    - Geography: 독일 고객의 이탈률 압도적으로 높음
    - Gender: 여성 고객의 이탈률 더 높음

- 중간 영향 변수
    - Card Type: 일부 차이 있으나 제한적

- 영향 적음
    - HasCrCard, Point Earned

In [34]:
df_clean.dtypes

CreditScore             int64
Geography              object
Gender                 object
Age                     int64
Tenure                  int64
Balance               float64
NumOfProducts           int64
HasCrCard               int64
IsActiveMember          int64
EstimatedSalary       float64
Exited                  int64
Complain                int64
Satisfaction Score      int64
Card Type              object
Point Earned            int64
NumOfProducts_         object
dtype: object

### feature 조합

- IsActiveMember × HasCrCard
    - 단독 HasCrCard는 영향이 없지만,
    - 비활성 + 신용카드 없음 조합은 이탈을 더 잘 설명할 수 있음.

- Geography × Gender  
    - 독일 + 여성 조합이 특히 이탈률이 높을 가능성.
    - 단순히 성별이나 국가 하나로 설명되지 않는 패턴을 잡을 수 있음.

- Satisfaction Score × Complain
    - 불만 제기는 강한 요인인데, 만족 점수가 낮은데 불만 제기를 안 한 경우도 있을 수 있음.
    - “숨은 불만 고객”을 찾는 데 유용할 수 있음.

- Balance × NumOfProducts
    - 잔액이 많고 상품 수가 많은 고객이 특히 이탈률이 높은지 확인.
    - 단독보다 interaction이 더 설명력이 클 수 있음.

- EstimatedSalary × Age
    - 젊은 고소득층 vs 중장년 저소득층 → 전혀 다른 churn 패턴 가능.

In [35]:
df_ = df_clean.copy()

# 안전한 형 변환
bin_cols = ['IsActiveMember','HasCrCard']
for c in bin_cols:
    df_[c] = df_[c].astype(str)

# ===== A. 기본 조합 (강한 변수 중심) =====

# 1) IsActiveMember × HasCrCard (참/거짓 조합)
df_['ia_x_card'] = df_['IsActiveMember'].astype(str) + '_' + df_['HasCrCard'].astype(str)

# 2) Geography × Gender (지역·성별 상호작용)
df_['geo_x_gender'] = df_['Geography'].astype(str) + '_' + df_['Gender'].astype(str)


# 3) Balance (bin) × NumOfProducts (bin)
# df_['bal_bin']  = pd.qcut(df_['Balance'], q=5, duplicates='drop')
# df_['prod_bin'] = pd.cut(df_['NumOfProducts'], bins=[0,1,2,3,4], right=True, include_lowest=True)
# df_['balbin_x_prodbin'] = df_['bal_bin'].astype(str) + '_' + df_['prod_bin'].astype(str)

# 4) Age (bin) × EstimatedSalary (bin)
df_['age_bin'] = pd.qcut(df_['Age'], q=5, duplicates='drop')
df_['sal_bin'] = pd.qcut(df_['EstimatedSalary'], q=5, duplicates='drop')
df_['agebin_x_salbin'] = df_['age_bin'].astype(str) + '_' + df_['sal_bin'].astype(str)

# ===== B. '약한 변수'들도 '강한 변수'와 결합하여 시도 =====

# 5) CreditScore (bin) × Geography
# df_['cs_bin'] = pd.qcut(df_['CreditScore'], q=5, duplicates='drop')
# df_['csbin_x_geo'] = df_['cs_bin'].astype(str) + '_' + df_['Geography'].astype(str)

# 6) Tenure (bin) × IsActiveMember
df_['ten_bin'] = pd.qcut(df_['Tenure'], q=5, duplicates='drop')
df_['tenbin_x_ia'] = df_['ten_bin'].astype(str) + '_' + df_['IsActiveMember'].astype(str)

# 7) Card Type × IsActiveMember  (카디널리티 낮아 안전)
df_['cardtype_x_ia'] = df_['Card Type'].astype(str) + '_' + df_['IsActiveMember'].astype(str)

# ===== C. 수치형 상호작용(트리/선형모델 모두에서 유효) =====
# 스케일 민감한 선형 모델을 쓸 땐 이후 표준화 권장
df_['age_x_balance'] = df_['Age'] * df_['Balance']
df_['age_x_products'] = df_['Age'] * df_['NumOfProducts']
df_['balance_x_products'] = df_['Balance'] * df_['NumOfProducts']
# df_['cs_x_balance'] = df_['CreditScore'] * df_['Balance']

# ===== D. 정리: 학습에 바로 넣을 때 불필요 임시 bin 컬럼을 남겨둘지 결정 =====
#  임시 bin들을 제거 시 아래 코드 활성화
df_.drop(columns=['sat_bin','age_bin','sal_bin','cs_bin','ten_bin','bin'], inplace=True, errors='ignore')


In [36]:
# 팀장님 feature 조합 코드
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()

    # 컬럼 명에 공백/대문자 등 정리 (선택)
    # 여기서는 원본 이름 유지. 접근 시 대괄호 인덱싱 사용: data['Satisfaction Score']

    # Age Group
    data['Age_Group'] = pd.cut(
        data['Age'],
        bins=[18, 30, 40, 50, 100],
        labels=['18-30', '31-40', '41-50', '51+'],
        include_lowest=True
    )

    # Senior Flag
    data['Senior_Flag'] = (data['Age'] >= 45).astype(int)

    # 독일 플래그 + 독일&고잔액 상호작용
    data['Germany_Flag'] = (data['Geography'] == 'Germany').astype(int)
    median_balance = data['Balance'].median()
    data['Germany_HighBalance'] = ((data['Geography'] == 'Germany') & (data['Balance'] > median_balance)).astype(int)

    # 잔액/상품수 (참여도 지표)
    data['Balance_per_Product'] = data['Balance'] / (data['NumOfProducts'].replace(0, np.nan))
    data['Balance_per_Product'] = data['Balance_per_Product'].fillna(0.0)

    # 비활동 & 상품1개 (고위험)
    data['LowActive_LowProduct'] = ((data['IsActiveMember'] == 0) & (data['NumOfProducts'] == 1)).astype(int)

    # 불만 & 활동 여부 결합
    # data['Complain_Active'] = data['Complain'].astype(str) + "_" + data['IsActiveMember'].astype(str)

    # 만족도 구간 (Low/Medium/High)
    data['Satisfaction_Level'] = pd.cut(
        data['Satisfaction Score'],
        bins=[0, 2, 4, 5],
        labels=['Low', 'Medium', 'High'],
        include_lowest=True
    )

    #complain 제거
    data = data.drop(columns=['Complain'])

    return data


In [37]:
df_ = engineer_features(df_)

#### 수치형 feature p-value 확인

In [53]:
import numpy as np
numeric_cols = df_.drop(columns=["Exited"]).select_dtypes(include=np.number).columns
numeric_cols

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'EstimatedSalary', 'Satisfaction Score', 'Point Earned',
       'age_x_balance', 'age_x_products', 'balance_x_products', 'Senior_Flag',
       'Germany_Flag', 'Germany_HighBalance', 'Balance_per_Product',
       'LowActive_LowProduct'],
      dtype='object')

In [39]:
# p-value 확인 : 각 수치형 feature는 유의미?
from scipy.stats import chi2_contingency

for col in numeric_cols:
    table = pd.crosstab(df_[col], df_['Exited'])
    chi2, p, dof, expected = chi2_contingency(table)
    print(col,":", p<0.05)

CreditScore : False
Age : True
Tenure : False
Balance : True
NumOfProducts : True
EstimatedSalary : False
Satisfaction Score : False
Point Earned : False
age_x_balance : True
age_x_products : True
balance_x_products : True
Senior_Flag : True
Germany_Flag : True
Germany_HighBalance : True
Balance_per_Product : True
LowActive_LowProduct : False


#### 유의미 수치형 feature : 
- Age, Balance, NumOfProducts, IsActiveMember, Complain
- IsActiveMember, Complain 범주형으로 변경 가능 feature

In [40]:
df_clean.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Complain', 'Satisfaction Score', 'Card Type', 'Point Earned',
       'NumOfProducts_'],
      dtype='object')

#### 범주형 feature p-value 확인

In [41]:
cat_cols = [col for col in df_.columns if col not in numeric_cols]

In [42]:
# p-value 확인 : 각 feature는 유의미?

for col in cat_cols:
    table = pd.crosstab(df_[col], df_['Exited'])
    chi2, p, dof, expected = chi2_contingency(table)
    print(col,":", p<0.05)

Geography : True
Gender : True
HasCrCard : False
IsActiveMember : True
Exited : True
Card Type : False
NumOfProducts_ : True
ia_x_card : True
geo_x_gender : True
agebin_x_salbin : True
tenbin_x_ia : True
cardtype_x_ia : True
Age_Group : True
Satisfaction_Level : False


#### 유의미 범주형 feature : 
- Geography , Gender , NumOfProducts_ , bin , Complain ,IsActiveMember
- NumOfProducts_ :  NumOfProducts의 범주형 feature화
- NumOfProducts_ : NumOfProducts과 크게 다르지 않음(=사용x)

In [43]:
true_cols = []
for col in df_.columns:
    table = pd.crosstab(df_[col], df_['Exited'])
    chi2, p, dof, expected = chi2_contingency(table)
    if p<0.05:
        true_cols.append(col)
true_cols

['Geography',
 'Gender',
 'Age',
 'Balance',
 'NumOfProducts',
 'IsActiveMember',
 'Exited',
 'NumOfProducts_',
 'ia_x_card',
 'geo_x_gender',
 'agebin_x_salbin',
 'tenbin_x_ia',
 'cardtype_x_ia',
 'age_x_balance',
 'age_x_products',
 'balance_x_products',
 'Age_Group',
 'Senior_Flag',
 'Germany_Flag',
 'Germany_HighBalance',
 'Balance_per_Product']

In [44]:
selected_features = [
    # 1. 원본
    "Geography",
    "Gender",
    "Age",
    "Balance",
    "NumOfProducts",
    "IsActiveMember",

    # 2. 파생/조합 (8개)
    "NumOfProducts_",
    "ia_x_card",
    "geo_x_gender",
    "balbin_x_prodbin",
    "agebin_x_salbin",
    "csbin_x_geo",
    "tenbin_x_ia",
    "cardtype_x_ia",

    # 3. 플래그/비율형 (2개)
    "Germany_Flag",
    "Balance_per_Product",

    # 4. 만족도
    "Satisfaction_Level"
]

### 라이브러리 선언

In [45]:
import numpy as np
import pandas as pd

# 모델 관련
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

# 데이터 전처리 & 분할
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 불균형 처리
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline

# 평가 지표
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    make_scorer
)

In [46]:
# === 0) 추천 컬럼(총 12개) ===
recommended_cols = [
    # 1) 원본 (6)
    "Geography", "Gender", "Age", "Balance", "NumOfProducts", "IsActiveMember",
    # 2) 파생/조합 (4)
    "ia_x_card", "geo_x_gender", "agebin_x_salbin", "cardtype_x_ia",
    # 3) 플래그/비율 (2)
    "Germany_Flag"
]

### catboost

In [47]:


df_use = df_.copy()
y = df_use["Exited"].astype(int)
X = df_use[recommended_cols].copy()

# CatBoost 범주형 인식: 문자열로 캐스팅
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype(str)
cat_idx = [X.columns.get_loc(c) for c in cat_cols]

# === 1) Train / Test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_pool = Pool(X_train, y_train, cat_features=cat_idx)
valid_pool = Pool(X_test,  y_test,  cat_features=cat_idx)

# === 2) CatBoost 학습 ===
model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    iterations=800,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3.0,
    auto_class_weights="Balanced",  # 클래스 불균형 자동 보정
    random_seed=42,
    verbose=100
)
model.fit(train_pool, eval_set=valid_pool, use_best_model=True, early_stopping_rounds=100)


# === 3) 성능 지표 ===
proba = model.predict_proba(valid_pool)[:, 1]
pred  = (proba >= 0.5).astype(int)

metrics = {
    "ACC"      : accuracy_score(y_test, pred),
    "F1"       : f1_score(y_test, pred),
    "Precision": precision_score(y_test, pred, zero_division=0),
    "Recall"   : recall_score(y_test, pred),
    "AUC"      : roc_auc_score(y_test, proba),
}
for k, v in metrics.items():
    print(f"{k:>9} : {v:.4f}")


0:	test: 0.8400774	best: 0.8400774 (0)	total: 238ms	remaining: 3m 10s
100:	test: 0.8768405	best: 0.8773001 (81)	total: 5.51s	remaining: 38.2s
200:	test: 0.8798488	best: 0.8798488 (200)	total: 10.3s	remaining: 30.8s
300:	test: 0.8790082	best: 0.8799166 (201)	total: 15.6s	remaining: 25.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8799165866
bestIteration = 201

Shrink model to first 202 iterations.
      ACC : 0.8045
       F1 : 0.6185
Precision : 0.5138
   Recall : 0.7770
      AUC : 0.8799


## Randomforest + SMOTENC

In [48]:
# === 0) 추천 컬럼 === (그대로)
recommended_cols = [
    "Geography","Gender","Age","Balance","NumOfProducts","IsActiveMember",
    "ia_x_card","geo_x_gender","agebin_x_salbin","cardtype_x_ia",
    "Germany_Flag"
]
df_use = df_.copy()
X = df_use[recommended_cols].copy()
y = df_use["Exited"].astype(int)

# === 1) split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# === 2) 범주형 열 식별 (원본 X 기준으로 일관성 유지) ===
cat_like = X.select_dtypes(include=["object","category"]).columns.tolist()
# 의미상 범주형 추가(필요시)
for c in ["IsActiveMember", "Satisfaction_Level"]:
    if c in X.columns and c not in cat_like:
        cat_like.append(c)
cat_like = list(dict.fromkeys(cat_like))
num_like = [c for c in X.columns if c not in cat_like]

# 이름 -> 인덱스 (원본 X 기준으로 고정)
cat_idx = [X.columns.get_loc(c) for c in cat_like]
num_idx = [X.columns.get_loc(c) for c in num_like]

# === 3) SMOTENC (train에만 적용) ===
smote = SMOTENC(categorical_features=cat_idx, random_state=42, k_neighbors=5)
X_res_np, y_res = smote.fit_resample(X_train.values, y_train.values)

# ndarray -> DataFrame 복원 (열 이름 유지)
X_res = pd.DataFrame(X_res_np, columns=X.columns)  # ← 원본 X의 컬럼 순서

# === 4) OneHotEncoder (fit on train_res, transform both train_res/test) ===
try:
    oh = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    oh = OneHotEncoder(handle_unknown="ignore", sparse=False)

# 인덱스 기반 ColumnTransformer
preproc_idx = ColumnTransformer(
    transformers=[
        ("cat", oh, cat_idx),
        ("num", "passthrough", num_idx),
    ],
    remainder="drop"
)

# 적합/변환
X_res_enc  = preproc_idx.fit_transform(X_res.values)     # SMOTE된 train
X_test_enc = preproc_idx.transform(X_test.values)        # 원본 test

# === 5) RF 학습 ===
rf = RandomForestClassifier(
    n_estimators=400, max_depth=None, min_samples_leaf=10,
    class_weight="balanced", n_jobs=-1, random_state=42
)
rf.fit(X_res_enc, y_res)

# # === 6) 평가 ===
# proba = rf.predict_proba(X_test_enc)[:, 1]
# pred  = (proba >= 0.5).astype(int)

# print("ACC       :", accuracy_score(y_test, pred))
# print("F1        :", f1_score(y_test, pred))
# print("Precision :", precision_score(y_test, pred, zero_division=0))
# print("Recall    :", recall_score(y_test, pred))
# print("AUC       :", roc_auc_score(y_test, proba))

# === 7) 5-Fold Stratified CV ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 파이프라인: SMOTENC → OneHot(인덱스) → RandomForest
cv_pipe = ImbPipeline(steps=[
    ("smote", SMOTENC(
        categorical_features=cat_idx,
        random_state=42,
        k_neighbors=5
    )),
    ("preproc", preproc_idx),
    ("rf", RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=10,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    ))
])

scoring = {
    "ACC": make_scorer(accuracy_score),
    "Precision": make_scorer(precision_score, zero_division=0),
    "Recall": make_scorer(recall_score),
    "F1": make_scorer(f1_score),
    "ROC_AUC": "roc_auc",  # 확률 필요: 파이프라인 내부에서 predict_proba 호출됨
}

cvres = cross_validate(
    cv_pipe,
    X.values, y.values,
    cv=cv, scoring=scoring,
    n_jobs=-1, return_train_score=False
)

def mstd(k): 
    return f"{cvres[f'test_{k}'].mean():.4f} ± {cvres[f'test_{k}'].std():.4f}"

print("\n==== 5-Fold CV (RF + SMOTENC + OneHot) ====")
print("ACC      :", mstd("ACC")),
print("F1       :", mstd("F1"))
print("Precision:", mstd("Precision"))
print("Recall   :", mstd("Recall"))
print("ROC_AUC  :", mstd("ROC_AUC"))



==== 5-Fold CV (RF + SMOTENC + OneHot) ====
ACC      : 0.8415 ± 0.0048
F1       : 0.6133 ± 0.0112
Precision: 0.6103 ± 0.0137
Recall   : 0.6168 ± 0.0193
ROC_AUC  : 0.8528 ± 0.0085


## XGBoost + 교차검증 + SMOTE

In [49]:
# ===== 1) 타깃/피처 분리 =====
TARGET_COL = "Exited"
y = df_[TARGET_COL].astype(int)

base_cols = [
    # 1) 원본 (6)
    "Geography","Gender","Age","Balance","NumOfProducts","IsActiveMember",
    # 2) 파생/조합 (4)
    "ia_x_card","geo_x_gender","agebin_x_salbin","cardtype_x_ia",
    # 3) 플래그/비율 + 만족도
    "Germany_Flag"
]
use_cols = [c for c in base_cols if c in df_.columns]
X = df_[use_cols].copy()

# ===== 2) 전처리: 인덱스 기반으로 설정 (SMOTENC 이후 ndarray 대응) =====
# 문자열/카테고리형으로 취급할 열 이름
cat_like = X.select_dtypes(include=["object","category"]).columns.tolist()

# 의미상 범주형(0/1 등)도 포함하고 싶다면 아래 추가
for c in ["IsActiveMember","Satisfaction_Level"]:
    if c in X.columns and c not in cat_like:
        # Satisfaction_Level이 이미 object면 자동 포함됨
        # IsActiveMember가 int형이면 의미상 범주로 포함
        cat_like.append(c)

# 중복 제거
cat_like = list(dict.fromkeys(cat_like))
num_like = [c for c in X.columns if c not in cat_like]

# 이름 → 인덱스
cat_idx = [X.columns.get_loc(c) for c in cat_like]
num_idx = [X.columns.get_loc(c) for c in num_like]

# OneHotEncoder: 신규 버전은 sparse_output, 구버전은 sparse
try:
    oh = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    oh = OneHotEncoder(handle_unknown="ignore", sparse=False)

preproc_idx = ColumnTransformer(
    transformers=[
        ("cat", oh, cat_idx),
        ("num", "passthrough", num_idx),
    ],
    remainder="drop"
)

# ===== 3) 모델 정의 =====
model = xgb.XGBClassifier(
    n_estimators=300, learning_rate=0.05, max_depth=5,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, eval_metric="logloss"
)

# ===== 4) SMOTENC (≈60:40) + 파이프라인 =====
smote = SMOTENC(
    categorical_features=cat_idx,
    sampling_strategy=0.67,  # ≈ 60:40
    random_state=42,
    k_neighbors=5
)

pipe = ImbPipeline(steps=[
    ("smote", smote),      # 각 fold의 train에만 합성
    ("preproc", preproc_idx),
    ("clf", model),
])

# ===== 5) 5-Fold 교차검증 =====
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "ACC": make_scorer(accuracy_score),
    "F1":  make_scorer(f1_score),
}

# ndarray로 넣어 인덱스 전처리와 일관성 확보
cvres = cross_validate(
    pipe, X.values, y.values,
    cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False
)

acc_mean, acc_std = cvres["test_ACC"].mean(), cvres["test_ACC"].std()
f1_mean,  f1_std  = cvres["test_F1"].mean(),  cvres["test_F1"].std()

print("==== 5-Fold CV (SMOTENC ~60:40, index-based) ====")
print(f"Accuracy : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1       : {f1_mean:.4f} ± {f1_std:.4f}")
print("Per-fold ACC:", np.round(cvres["test_ACC"], 4))
print("Per-fold F1 :", np.round(cvres["test_F1"], 4))

==== 5-Fold CV (SMOTENC ~60:40, index-based) ====
Accuracy : 0.8591 ± 0.0063
F1       : 0.6186 ± 0.0114
Per-fold ACC: [0.853  0.8575 0.854  0.8705 0.8605]
Per-fold F1 : [0.607  0.6185 0.6107 0.6398 0.6173]


In [50]:
==== 5-Fold CV (SMOTENC ~60:40, index-based) ====
Accuracy : 0.8561 ± 0.0074
F1       : 0.6159 ± 0.0229
Per-fold ACC: [0.8502 0.8465 0.8605 0.8563 0.8672]
Per-fold F1 : [0.6057 0.5795 0.6184 0.6272 0.6484]

SyntaxError: invalid character '±' (U+00B1) (1331683612.py, line 2)

## Catboost + 교차검증 + SMOTE

In [51]:
# 추천/상호작용 피처
recommended_cols = [
     # 1. 원본 (6개)
    "Geography",
    "Gender",
    "Age",
    "Balance",
    "NumOfProducts",
    "IsActiveMember",

    # 2. 파생/조합 (4개)
    "ia_x_card",
    "geo_x_gender",
    "agebin_x_salbin",
    "cardtype_x_ia",

    # 3. 플래그 (2개)
    "Germany_Flag",
]
X = df_[ [c for c in recommended_cols if c in df_.columns] ].copy()
y = df_["Exited"].astype(int).values

# CatBoost에서 범주형으로 처리할 열
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
cat_idx  = [X.columns.get_loc(c) for c in cat_cols]

# 안전: CatBoost는 문자열/카테고리형을 권장
for c in cat_cols:
    X[c] = X[c].astype(str)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_folds_catboost(X_df: pd.DataFrame, y_arr: np.ndarray, model_kwargs: dict,
                            use_smote: bool = False, sampling_strategy: float = 0.67):
    accs, f1s, precs, recs, aucs = [], [], [], [], []

    # 최적 임계값 저장용
    oof_proba = np.zeros(len(y))
    oof_true  = np.zeros(len(y))

    for tr_idx, te_idx in cv.split(X_df, y_arr):
        X_tr, X_te = X_df.iloc[tr_idx].copy(), X_df.iloc[te_idx].copy()
        y_tr, y_te = y_arr[tr_idx], y_arr[te_idx]

        # (B) SMOTENC 적용 옵션
        if use_smote:
            # SMOTENC는 ndarray를 반환 → DataFrame으로 복원
            smote = SMOTENC(categorical_features=cat_idx,
                            sampling_strategy=sampling_strategy,
                            random_state=42, k_neighbors=5)
            X_res, y_res = smote.fit_resample(X_tr.values, y_tr)
            X_tr = pd.DataFrame(X_res, columns=X_df.columns)
            y_tr = y_res
            # 범주형을 다시 문자열로 보장
            for c in cat_cols:
                X_tr[c] = X_tr[c].astype(str)
            for c in cat_cols:
                X_te[c] = X_te[c].astype(str)

        # CatBoost Pool 생성
        train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
        test_pool  = Pool(X_te, y_te, cat_features=cat_idx)

        # 모델 학습 (early stopping 포함)
        model = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            iterations=800,
            learning_rate=0.05,
            depth=6,
            l2_leaf_reg=3.0,
            random_state=42,
            verbose=False,
            **model_kwargs
        )
        model.fit(train_pool, eval_set=test_pool, use_best_model=True, early_stopping_rounds=100, verbose=False)

        # 예측/지표
        proba = model.predict_proba(test_pool)[:, 1]
        oof_proba[te_idx] = proba
        oof_true[te_idx] = y_te

        pred  = (proba >= 0.39).astype(int)  # 필요 시 threshold 최적화 가능

        accs.append(accuracy_score(y_te, pred))
        f1s.append(f1_score(y_te, pred))
        precs.append(precision_score(y_te, pred, zero_division=0))
        recs.append(recall_score(y_te, pred))
        aucs.append(roc_auc_score(y_te, proba))

    best_th, best_f1 = 0.5,0
    for th in np.linspace(0.2,0.8,61):
        f1 = f1_score(oof_true,(oof_proba >= th).astype(int))
        if f1 > best_f1:
            best_f1, best_th = f1,th

    print(f"Best F1={best_f1:.4f} at threshold={best_th:.3f}")  

    def fmt(arr): 
        return f"{np.mean(arr):.4f} ± {np.std(arr):.4f}"
    report = {
    "ACC": fmt(accs), "F1": fmt(f1s), "Precision": fmt(precs),
    "Recall": fmt(recs), "ROC_AUC": fmt(aucs)
    }
    return report, best_th

# CatBoost + SMOTENC(≈60:40) 5-Fold 
res_smote,best_th = evaluate_folds_catboost(
    X, y,
    model_kwargs=dict(auto_class_weights=None),
    use_smote=True,
    sampling_strategy=0.67  # ≈ 60:40
)
print("\n=== CatBoost + SMOTENC(≈60:40) 5-Fold ===")
for k, v in res_smote.items():
    print(f"{k:>9} : {v}")
print(f"[OOF] Best threshold = {best_th:.3f}")


Best F1=0.6337 at threshold=0.370

=== CatBoost + SMOTENC(≈60:40) 5-Fold ===
      ACC : 0.8481 ± 0.0060
       F1 : 0.6304 ± 0.0107
Precision : 0.6257 ± 0.0170
   Recall : 0.6354 ± 0.0080
  ROC_AUC : 0.8708 ± 0.0072
[OOF] Best threshold = 0.370


In [52]:
# === 추천/상호작용 피처 ===
recommended_cols = [
    # 1) 원본 (6)
    "Geography","Gender","Age","Balance","NumOfProducts","IsActiveMember",
    # 2) 파생/조합 (4)
    "ia_x_card","geo_x_gender","agebin_x_salbin","cardtype_x_ia",
    # 3) 플래그 (1)
    "Germany_Flag",
]

X = df_[ [c for c in recommended_cols if c in df_.columns] ].copy()
y = df_["Exited"].astype(int).values

# CatBoost 범주형 지정
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype(str)
cat_idx  = [X.columns.get_loc(c) for c in cat_cols]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_folds_catboost_balanced(X_df: pd.DataFrame, y_arr: np.ndarray):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    accs, f1s, precs, recs, aucs = [], [], [], [], []

    # OOF 저장
    oof_proba = np.zeros(len(y_arr))
    oof_true  = np.zeros(len(y_arr))

    for tr_idx, te_idx in cv.split(X_df, y_arr):
        X_tr, X_te = X_df.iloc[tr_idx].copy(), X_df.iloc[te_idx].copy()
        y_tr, y_te = y_arr[tr_idx], y_arr[te_idx]

        # 범주형 안전 캐스팅
        for c in cat_cols:
            X_tr[c] = X_tr[c].astype(str)
            X_te[c] = X_te[c].astype(str)

        # CatBoost Pool
        train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
        test_pool  = Pool(X_te, y_te, cat_features=cat_idx)

        # CatBoost (SMOTE 없이, 클래스 가중치 자동)
        model = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            iterations=800,
            learning_rate=0.05,
            depth=6,
            l2_leaf_reg=3.0,
            auto_class_weights="Balanced",
            random_state=42,
            verbose=False
        )
        model.fit(train_pool, eval_set=test_pool, use_best_model=True, early_stopping_rounds=100, verbose=False)

        # 예측/지표 (per-fold 기준 임계값 0.5)
        proba = model.predict_proba(test_pool)[:, 1]
        pred  = (proba >= 0.62).astype(int)

        oof_proba[te_idx] = proba
        oof_true[te_idx]  = y_te

        accs.append(accuracy_score(y_te, pred))
        f1s.append(f1_score(y_te, pred))
        precs.append(precision_score(y_te, pred, zero_division=0))
        recs.append(recall_score(y_te, pred))
        aucs.append(roc_auc_score(y_te, proba))

    # OOF 기준 최적 threshold 탐색
    best_th, best_f1 = 0.5, 0.0
    for th in np.linspace(0.2, 0.8, 61):
        f1 = f1_score(oof_true, (oof_proba >= th).astype(int))
        if f1 > best_f1:
            best_f1, best_th = f1, th

    print(f"[OOF] Best F1 = {best_f1:.4f} at threshold = {best_th:.3f}")

    def fmt(arr): return f"{np.mean(arr):.4f} ± {np.std(arr):.4f}"
    report = {"ACC": fmt(accs), "F1": fmt(f1s), "Precision": fmt(precs), "Recall": fmt(recs), "ROC_AUC": fmt(aucs)}
    return report, best_th

# === 실행 ===
res_balanced, best_th = evaluate_folds_catboost_balanced(X, y)
print("\n=== CatBoost (auto_class_weights='Balanced', no SMOTE) 5-Fold ===")
for k, v in res_balanced.items():
    print(f"{k:>9} : {v}")
print(f"[OOF] Best threshold = {best_th:.3f}")


[OOF] Best F1 = 0.6343 at threshold = 0.620

=== CatBoost (auto_class_weights='Balanced', no SMOTE) 5-Fold ===
      ACC : 0.8487 ± 0.0062
       F1 : 0.6344 ± 0.0119
Precision : 0.6254 ± 0.0166
   Recall : 0.6438 ± 0.0114
  ROC_AUC : 0.8708 ± 0.0062
[OOF] Best threshold = 0.620
